# Laboratorio 2 - Reglas de Asociación y Patrones Secuenciales

## Grupo 1 - Paula Hípola Gómez, José Ignacio Navas Sanz y Belén Ortega Pérez

## Entregable 1 - Reglas de Asociación

### 1.1 (1 punto) Estudia brevemente el dataset (número de usuarios, número de atributos, categorías más frecuentes para cada atributo/columna, etc.) y realiza el preprocesamiento para generar el dataset necesario para aplicar el algoritmo Apriori a este conjunto de datos.

In [2]:
%config IPCompleter.greedy=True
%reset
#Importamos las librerías necesarias
#Numpy
import numpy as np 
#Pandas
import pandas as pd
#Scikit-Learn
import sklearn as sk
#Matplotlib
import matplotlib.pyplot as plt
#Importamos la libreria mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


### Cargamos los datos y los visualizamos

In [3]:
#Importamos el CSV
dataset = pd.read_csv('radio.csv')

#Mostramos los datos
dataset.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [4]:
print('La dimensión del dataset es:',dataset.shape)

La dimensión del dataset es: (289955, 4)


Tenemos 289955 instancias y 4 atributos, para cada una de nuestras instancias. Analizando más profundamente el dataset, está compuesto con las interacciones de cada usuario que ha escuchado al menos a un artista. Vamos a ver cuantos valores tenemos actualmente en cuanto a usuarios, artistas y demás atributos y luego procederemos a identificar en el dataset valores duplicados y nulos que dificultan la comprensión de los datos y procederemos a su eliminación.

In [5]:
#Valores desconocidos
#dataset.loc[dataset['artist'] == '[unknown]']
#dataset = dataset.replace('[unknown]',np.NaN)

### Limpieza de datos

In [6]:
#Eliminación de nulos
dataset = dataset.dropna()
print('Tamaño sin nulos',dataset.shape)

#Eliminación de duplicados
dataset = dataset.drop_duplicates()
print('Tamaño sin duplicados',dataset.shape)
dataset

Tamaño sin nulos (289955, 4)
Tamaño sin duplicados (289953, 4)


,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
289950,19718,bob dylan,f,Canada
289951,19718,pixies,f,Canada
289952,19718,the clash,f,Canada
289953,19718,a tribe called quest,f,Canada


In [7]:
numUsuarios = dataset.groupby('user').count().shape[0]
print('Hay',numUsuarios,'usuarios.')
numArtistas = dataset.groupby('artist').count().shape[0]
print('Hay',numArtistas,'artistas.')
numSexos = dataset.groupby('sex').count().shape[0]
print('Hay',numSexos,'géneros.')
numPaises = dataset.groupby('country').count().shape[0]
print('Hay',numPaises,'países.')

Hay 15000 usuarios.
Hay 1004 artistas.
Hay 2 géneros.
Hay 159 países.


In [8]:
dataset['country'].describe()

count            289953
unique              159
top       United States
freq              59558
Name: country, dtype: object

In [9]:
dataset['artist'].describe()

count        289953
unique         1004
top       radiohead
freq           2704
Name: artist, dtype: object

Como podemos ver, hemos agrupado todos los usuarios, artistas, géneros y países en sus respectivas categorías para comprender mejor el dataset planteado, ahora procederemos a preprocesar y limpiar los datos. Si analizamos este dataset, vemos que tenemos 15000 usuarios si bien este no es un dato real puesto que hay saltos en el ID de usuario. Esto se debe a que tenemos los datos de los usuarios que han escuchado al menos 1 artista por lo que no podemos sacar una conclusión del número exacto de usuarios, si no de que hay 15000 usuarios que han escuchado al menos a un artista.

Tenemos que hacer un nuevo dataset para analizar estos datos con el algoritmo Apriori, para ello vamos a construir un dataset con un listado de ususarios con todos los artistas que ha escuchado...

In [10]:
dagrupado = dataset.groupby('user')['artist'].apply(list)
dfAgrupado = pd.DataFrame(dagrupado)
dfAgrupado

,artist
user,
1,"[red hot chili peppers, the black dahlia murde..."
3,"[devendra banhart, boards of canada, cocorosie..."
4,"[tv on the radio, tool, kyuss, dj shadow, air,..."
5,"[dream theater, ac/dc, metallica, iron maiden,..."
6,"[lily allen, kanye west, sigur rós, pink floyd..."
...,...
19713,"[armin van buuren, above & beyond, atb, ferry ..."
19714,"[misfits, type o negative, arch enemy, red hot..."
19715,"[abba, james blunt, jason mraz, amy winehouse,..."


Pese a tener los datos del país del usuario además de su género, nos damos cuenta de que estos datos carecen de valor y no aportan nada al objetivo que perseguimos, que no es más que la adquisición de los conjuntos de grupos de música más frecuentes para la obtención de reglas de asociación que nos permita el estudio de estos grupos y sus similitudes por el cual se produzcan estos agrupamientos.

## 1.2 (4 puntos) Aprende a manejar la librería mlextend y su implementación del algoritmo Apriori.

In [11]:
te = TransactionEncoder()
te_ary = te.fit(dagrupado).transform(dagrupado)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14997,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
14998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### 1.2.1 Obtén los itemsets frecuentes para k=1. Para esto es necesario obtener el soporte de los itemset. Crea una función que utilice métodos de la librería mlextend de manera que dado un itemset devuelva su soporte.

#### La línea de a continuación tarda un rato, está ejecutando que no roto.

In [12]:
#Necesitamos definir un soporte minimo. sabemos que tenemos 14995 instancias, por lo que consideramos que todo lo que este
#por encima de... 250/14995 será para nosotros válido a la hora de operar con ello
frequent_itemsets = apriori(df, min_support=0.010, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets1 = frequent_itemsets[frequent_itemsets['length'] == 1]

In [13]:
frequent_itemsets1.reset_index()

,index,support,itemsets,length
0,0,0.022733,(2pac),1
1,1,0.030933,(3 doors down),1
2,2,0.032800,(30 seconds to mars),1
3,3,0.021800,(50 cent),1
4,4,0.013667,(65daysofstatic),1
...,...,...,...,...
650,650,0.012267,(wilco),1
651,651,0.011933,(within temptation),1
652,652,0.019600,(yann tiersen),1
653,653,0.017333,(yeah yeah yeahs),1


In [14]:
artistas = [];
def soporteItemset(itemset):
    #Obtengo una lista de artistas
    for x in frequent_itemsets1['itemsets']:
        artista = (list(x))
        artistas.append(artista)
    #Recorro la lista de artistas
    for i in range(len(artistas)):
        if itemset in artistas[i]:
            soporte = frequent_itemsets1.loc[i]['support']
            return 'El sporte para ' + str(itemset) + ' es de: ' + str(round(soporte,4))
    return 'El artista ' + str(itemset) + ' no está en el itemset'

In [15]:
coldplay = soporteItemset('coldplay')
print(coldplay)

ecdl = soporteItemset('el canto del loco')
print(ecdl)

beatles = soporteItemset('the beatles')
print(beatles)

El sporte para coldplay es de: 0.1585
El artista el canto del loco no está en el itemset
El sporte para the beatles es de: 0.1779


#### 1.2.2. Crea una función que devuelva los itemsets frecuentes candidatos y su soporte para k>=2



Creamos una función que nos irá calculando itemsets frecuentes candidatos a medida que sea posible para k>=2 y que parará cuando no se generen estos itemsets. Esta función nos sirve además para el siguiente apartado donde se pide más o menos lo mismo

In [16]:
def soporteItemsetKMayorIgual(k):
    #Creamos un diccionario. Esto servira para concatenarlo al nombre del itemset generado
    valoresValidosK = []
    data = dict();
    numCreados = 0;
    if k < 2:
        return 'La k debe ser mayor o igual a 2'
    else:
        while k != -1:
            print('Creando para k = ',k)
            frq = frequent_itemsets[frequent_itemsets['length'] == k]
            if(frq.shape[0] > 0):
                print(frq.shape[0])
                valoresValidosK.append(k);
                var = 'frq'+str(k);
                print(var,'Creado')
                print('Aumentamos k\n')
                k += 1;
                numCreados += 1;
            else:
                print('No se puede generar un itemset para k = ',k)
                print('Paramos el bucle. No generamos nada para k >= ',k)
                k = -1;
        for i in range(numCreados):
            data['frq'+str(valoresValidosK[i])] = frequent_itemsets[frequent_itemsets['length'] == valoresValidosK[i]]        
        print('\nPara obtener los itemsets creados consulte el diccionario que devuelve la funcion.')
        print('Se han generado',numCreados,'datasets')
        return data,numCreados

In [17]:
itemsets, itemSetsGenerados = soporteItemsetKMayorIgual(2)

Creando para k =  2
962
frq2 Creado
Aumentamos k

Creando para k =  3
72
frq3 Creado
Aumentamos k

Creando para k =  4
No se puede generar un itemset para k =  4
Paramos el bucle. No generamos nada para k >=  4

Para obtener los itemsets creados consulte el diccionario que devuelve la funcion.
Se han generado 2 datasets


#### 1.2.3. Crea una función que repita el paso 1.2.2. hasta que no se generen nuevos itemsets frecuentes



La función que hemos generado en el paso 1.2.2 ya nos hace automáticamente esta función. Es decir, hemos creado una función que nos genera automáticamente los itemsets para k>=2 y que para cuando no hay itemsets frecuentes, por tanto no es necesario volver a repetir esta función

In [18]:
k2 = pd.DataFrame.from_dict(itemsets['frq2'])
k2

,support,itemsets,length
655,0.011067,"(linkin park, 3 doors down)",2
656,0.012533,"(linkin park, 30 seconds to mars)",2
657,0.011333,"(muse, a perfect circle)",2
658,0.010333,"(a perfect circle, nine inch nails)",2
659,0.013467,"(radiohead, a perfect circle)",2
...,...,...,...
1612,0.011200,"(the rolling stones, the white stripes)",2
1613,0.012733,"(the rolling stones, the who)",2
1614,0.012533,"(the rolling stones, u2)",2
1615,0.010133,"(the white stripes, the smashing pumpkins)",2


In [19]:
k3 = pd.DataFrame.from_dict(itemsets['frq3'])
k3

,support,itemsets,length
1617,0.011600,"(arctic monkeys, muse, coldplay)",3
1618,0.013067,"(arctic monkeys, radiohead, coldplay)",3
1619,0.012333,"(the killers, arctic monkeys, coldplay)",3
1620,0.011000,"(arctic monkeys, muse, radiohead)",3
1621,0.011067,"(the killers, arctic monkeys, muse)",3
...,...,...,...
1684,0.010600,"(the rolling stones, the beatles, radiohead)",3
1685,0.010667,"(the beatles, the shins, radiohead)",3
1686,0.011467,"(the beatles, the smashing pumpkins, radiohead)",3
1687,0.010467,"(the strokes, the beatles, radiohead)",3


#### 1.2.4. Crea una función que muestre todas las reglas posibles con su confianza



In [20]:
def sacarReglas(itemsets):
    reglas = association_rules(itemsets, support_only=False, metric="confidence", min_threshold=0.015)
    reglas = reglas[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'lift', 'confidence']]
    return reglas
sacarReglas(frequent_itemsets)

,antecedents,consequents,antecedent support,consequent support,support,lift,confidence
0,(linkin park),(3 doors down),0.098200,0.030933,0.011067,3.643163,0.112695
1,(3 doors down),(linkin park),0.030933,0.098200,0.011067,3.643163,0.357759
2,(linkin park),(30 seconds to mars),0.098200,0.032800,0.012533,3.891179,0.127631
3,(30 seconds to mars),(linkin park),0.032800,0.098200,0.012533,3.891179,0.382114
4,(muse),(a perfect circle),0.114067,0.036733,0.011333,2.704821,0.099357
...,...,...,...,...,...,...,...
2351,"(the white stripes, radiohead)",(the beatles),0.027267,0.177867,0.012267,2.529298,0.449878
2352,"(the beatles, radiohead)",(the white stripes),0.058200,0.069200,0.012267,3.045773,0.210767
2353,(the white stripes),"(the beatles, radiohead)",0.069200,0.058200,0.012267,3.045773,0.177264
2354,(the beatles),"(the white stripes, radiohead)",0.177867,0.027267,0.012267,2.529298,0.068966


#### 1.2.5. Lista todas las reglas que sean de alta confianza (p.ej. con valor >=0.5)



In [33]:
reglasAltaConfi = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
reglasAltaConfi = reglasAltaConfi[['antecedents', 'consequents', 'confidence', 'lift']]

In [34]:
reglasAltaConfi

,antecedents,consequents,confidence,lift
0,(beck),(radiohead),0.509281,2.825152
1,(blur),(radiohead),0.522863,2.900496
2,(broken social scene),(radiohead),0.547215,3.035589
3,(keane),(coldplay),0.637405,4.020634
4,(snow patrol),(coldplay),0.525132,3.312441
5,(the fray),(coldplay),0.516820,3.260006
6,(travis),(coldplay),0.562842,3.550304
7,(judas priest),(iron maiden),0.507500,8.562992
8,(t.i.),(kanye west),0.567273,8.854413
9,(megadeth),(metallica),0.528139,4.743759


#### 1.2.6. Crea funciones que permitan lo siguiente:
<ul>
    <li><b>* devolver todas las reglas que contengan un antecesor dado</b></li>
    <li><b>* devolver todas las reglas que tengan una confianza mayor (o igual) que un umbral mínimo dado</b></li>
    <li><b>* devolver todas las reglas que tengan un lift en una de las tres franjas de estudio (under 1, over 1, =1)</b></li>
</ul>



In [23]:
def reglasAntecesor(antecesor):
    antecedents = reglasAltaConfi['antecedents']
    index = antecedents.where([bool(antecesor in a) for a in list(antecedents)]).dropna().index
    return reglasAltaConfi.loc[index, :]

In [28]:
reglasAntecesor('the killers')

,antecedents,consequents,confidence,lift
16,"(the killers, bloc party)",(coldplay),0.523659,3.303150
20,"(the killers, death cab for cutie)",(coldplay),0.588448,3.711823
21,"(the killers, muse)",(coldplay),0.508969,3.210483
24,"(the killers, oasis)",(coldplay),0.662698,4.180183
27,"(the killers, radiohead)",(coldplay),0.524362,3.307582
30,"(the killers, red hot chili peppers)",(coldplay),0.509375,3.213047
31,"(the killers, snow patrol)",(coldplay),0.595420,3.755802
32,"(the killers, the beatles)",(coldplay),0.534091,3.368950


In [29]:
def reglasConfianza(confianza):
    confidence = reglasAltaConfi['confidence']
    index = confidence.where(confidence >= confianza).dropna().index
    return reglasAltaConfi.loc[index, :] 

In [30]:
reglasConfianza(0.6)

,antecedents,consequents,confidence,lift
3,(keane),(coldplay),0.637405,4.020634
17,"(bob dylan, pink floyd)",(the beatles),0.615079,3.458092
24,"(the killers, oasis)",(coldplay),0.662698,4.180183
26,"(radiohead, snow patrol)",(coldplay),0.634454,4.002021
28,"(the smashing pumpkins, coldplay)",(radiohead),0.628352,3.485683
45,"(the beatles, sigur rós)",(radiohead),0.643443,3.569393
48,"(the beatles, the smashing pumpkins)",(radiohead),0.620939,3.444556


In [31]:
def reglasLift(lft):
    lift = reglasAltaConfi['lift']
    
    if lft == 1: 
        index = lift.where(lift < lft).dropna().index
        print(reglasAltaConfi.loc[index, :])
    elif lft > 1:
        index = lift.where(lift > lft).dropna().index
        print(reglasAltaConfi.loc[index, :])
    elif lft < 1:
        index = lift.where(lift == lft).dropna().index
        print(reglasAltaConfi.loc[index, :])
    
    return reglasAltaConfi.loc[index, :]

In [32]:
reglasConfianza(0.6)

,antecedents,consequents,confidence,lift
3,(keane),(coldplay),0.637405,4.020634
17,"(bob dylan, pink floyd)",(the beatles),0.615079,3.458092
24,"(the killers, oasis)",(coldplay),0.662698,4.180183
26,"(radiohead, snow patrol)",(coldplay),0.634454,4.002021
28,"(the smashing pumpkins, coldplay)",(radiohead),0.628352,3.485683
45,"(the beatles, sigur rós)",(radiohead),0.643443,3.569393
48,"(the beatles, the smashing pumpkins)",(radiohead),0.620939,3.444556


#### 1.2.7. Utiliza al menos dos representaciones gráficas para mostrar las reglas obtenidas e interpretar los datos